In [71]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [72]:
import json
import os
import numpy as np
from utils import *
import math
import heapq

In [73]:
fd_collection = getCollection("team_5_mwdb_phase_2", "fd_collection")
all_images = fd_collection.find()

In [74]:
selected_latent_space = valid_latent_spaces[
    str(input("Enter latent space - one of " + str(list(valid_latent_spaces.keys()))))
]

selected_feature_model = valid_feature_models[
    str(input("Enter feature model - one of " + str(list(valid_feature_models.keys()))))
]

k = int(input("Enter value of k: "))
if k < 1:
    raise ValueError("k should be a positive integer")

selected_dim_reduction_method = str(
    input(
        "Enter dimensionality reduction method - one of "
        + str(list(valid_dim_reduction_methods.keys()))
    )
)

label = int(input("Enter label: "))
if label < 0 and label > 100:
    raise ValueError("label should be between 0 and 100")

knum = int(input("Enter value of knum: "))
if knum < 1:
    raise ValueError("knum should be a positive integer")

label_rep = calculate_label_representatives(fd_collection, label, selected_feature_model)

match selected_latent_space:
    case "":
        if selected_dim_reduction_method == "lda":
            if os.path.exists(f"{selected_feature_model}-{selected_dim_reduction_method}-{k}-model.joblib") and os.path.exists(f"{selected_feature_model}-{selected_dim_reduction_method}-{k}-semantics.json"):
                if os.path.exists(f"{selected_feature_model}-{selected_dim_reduction_method}-{k}-model.joblib"):
                    model = load(f"{selected_feature_model}-{selected_dim_reduction_method}-{k}-model.joblib")
                    data = json.load(open(f"{selected_feature_model}-{selected_dim_reduction_method}-{k}-semantics.json"))
                    print(f"{selected_feature_model}-{selected_dim_reduction_method}-{k}-model.joblib and json loaded")
                else:
                    print(f"{selected_feature_model}-{selected_dim_reduction_method}-{k}-model.joblib does not exist")
            else:
                if os.path.exists(f"{selected_feature_model}-{selected_dim_reduction_method}-{k}-semantics.json"):
                    data = json.load(open(f"{selected_feature_model}-{selected_dim_reduction_method}-{k}-semantics.json"))
                    print(f"{selected_feature_model}-{selected_dim_reduction_method}-{k}-semantics.json loaded")
                else:
                    print(f"{selected_feature_model}-{selected_dim_reduction_method}-{k}-semantics.json does not exist")
    case "cp":
        if os.path.exists(f"{selected_feature_model}-cp-{k}-semantics.json"):
            data = json.load(open(f"{selected_feature_model}-cp-{k}-semantics.json"))
            print(f"{selected_feature_model}-cp-{k}-semantics.json loaded")
        else:
          print(f"{selected_feature_model}-cp-{k}-semantics.json does not exist")
    case _:
        if selected_dim_reduction_method == "lda":
            if os.path.exists(f"{selected_latent_space}-{selected_feature_model}-{selected_dim_reduction_method}-{k}-model.joblib") and os.path.exists(f"{selected_latent_space}-{selected_feature_model}-{selected_dim_reduction_method}-{k}-semantics.json"):
                model = load(f"{selected_latent_space}-{selected_feature_model}-{selected_dim_reduction_method}-{k}-model.joblib")
                data = json.load(open(f"{selected_latent_space}-{selected_feature_model}-{selected_dim_reduction_method}-{k}-semantics.json"))
                print(f"{selected_latent_space}-{selected_feature_model}-{selected_dim_reduction_method}-{k}-model.joblib loaded")
            else:
                print(f"{selected_latent_space}-{selected_feature_model}-{selected_dim_reduction_method}-{k}-model.joblib does not exist")
        else:
            if os.path.exists(f"{selected_latent_space}-{selected_feature_model}-{selected_dim_reduction_method}-{k}-semantics.json"):
                data = json.load(open(f"{selected_latent_space}-{selected_feature_model}-{selected_dim_reduction_method}-{k}-semantics.json"))
                print(f"{selected_latent_space}-{selected_feature_model}-{selected_dim_reduction_method}-{k}-semantics.json loaded")
            else:
                print(f"{selected_latent_space}-{selected_feature_model}-{selected_dim_reduction_method}-{k}-semantics.json does not exist")

image_sim-cm_fd-lda-10-model.joblib loaded


In [75]:
def extract_similarities_ls1_ls4(latent_space, dim_reduction, data, label, label_rep):

  match dim_reduction:

    case 'svd':
      U = np.array(data["image-semantic"])
      S = np.array(data["semantics-core"])
      if len(S.shape) == 1:
        S = np.diag(S)
      V = np.transpose(np.array(data["semantic-feature"]))

      if latent_space == "image_sim":
        label_vectors = []
        length = len(U)
        for i in range(length):
          if all_images[i]["true_label"] == label:
            label_vectors.append(U[i])
        label_rep = [sum(col) / len(col) for col in zip(*label_vectors)]
      
      comparison_feature_space = np.matmul(U, S)

      if latent_space == "image_sim":
        comparison_vector = np.matmul(label_rep, S)
      else:
        comparison_vector = np.matmul(np.matmul(label_rep, V), S)
    
    case "nmf":
      H = np.array(data['semantic-feature'])
      comparison_feature_space = W = np.array(data['image-semantic'])
      if latent_space == "image_sim":
        label_vectors = []
        length = len(W)
        for i in range(length):
          if all_images[i]["true_label"] == label:
            label_vectors.append(W[i])
        label_rep = [sum(col) / len(col) for col in zip(*label_vectors)]

      if latent_space == "image_sim":
        comparison_vector = label_rep
      else:
        comparison_vector = np.matmul(label_rep, np.transpose(H))

    case "kmeans":
      comparison_vector = []
      comparison_feature_space = np.array(data["image-semantic"])
      S = np.array(data["semantic-feature"])

      if latent_space == "image_sim":
        sim_matrix = np.array(data["sim-matrix"])
        label_vectors = []
        length = len(sim_matrix)
        for i in range(length):
          if all_images[i]["true_label"] == label:
            label_vectors.append(sim_matrix[i])
        label_rep = [sum(col) / len(col) for col in zip(*label_vectors)]

      for centroid in S:
        comparison_vector.append(math.dist(label_rep, centroid))

    case "lda":
      comparison_feature_space = np.array(data["image-semantic"])
      label_vectors = []
      length = len(comparison_feature_space)
      for i in range(length):
        if all_images[i]["true_label"] == label:
          label_vectors.append(comparison_feature_space[i])
      comparison_vector = [sum(col) / len(col) for col in zip(*label_vectors)]      

  n = len(comparison_feature_space)

  distances = []
  for i in range(n):
    if all_images[i]["true_label"] != label:
      distances.append({"image_id": i, "label": all_images[i]["true_label"], "distance": math.dist(comparison_vector, comparison_feature_space[i])})

  distances = sorted(distances, key=lambda x: x["distance"], reverse=False)

  similar_labels = []
  unique_labels = set()

  for img in distances:
    if img['label'] not in unique_labels:
      similar_labels.append(img)
      unique_labels.add(img["label"])

      if len(similar_labels) == knum:
        break


  for x in similar_labels:
    print(x)

In [76]:
def extract_similarities_ls2(data, label):

  LS = np.array(data["label-semantic"])
  S = np.array(data["semantics-core"])

  if len(S.shape) == 1:
    S = np.diag(S)

  comparison_feature_space = np.matmul(LS, S)
  comparison_vector = comparison_feature_space[label]

  distances = []

  n = len(comparison_feature_space)
  for i in range(n):
    if i != label:
      distances.append({"label": i, "distance": math.dist(comparison_vector, comparison_feature_space[i])})
  
  distances = sorted(distances, key=lambda x: x["distance"], reverse=False)[:knum]

  for x in distances:
    print(x)

In [77]:
def extract_similarities_ls3(dim_reduction, data, label):

  match dim_reduction:

    case 'svd':
      U = np.array(data["image-semantic"])
      S = np.array(data["semantics-core"])
      V = np.transpose(np.array(data["semantic-feature"]))

      comparison_feature_space = np.matmul(U, S)
      comparison_vector = comparison_feature_space[label]
    
    case "nmf":
      comparison_feature_space = np.array(data['image-semantic'])
      comparison_vector = comparison_feature_space[label]

    case "kmeans":
      comparison_feature_space = np.array(data["image-semantic"])
      comparison_vector = comparison_feature_space[label]

    case "lda":
      comparison_feature_space = np.array(data["image-semantic"])
      comparison_vector = comparison_feature_space[label]      


  n = len(comparison_feature_space)
  distances = []
  for i in range(n):
    if i != label:
      distances.append({"label": i, "distance": math.dist(comparison_vector, comparison_feature_space[i])})

  distances = sorted(distances, key=lambda x: x["distance"], reverse=False)[:knum]

  for x in distances:
    print(x)

In [78]:
match selected_latent_space:

  case "" | "image_sim":
    
    extract_similarities_ls1_ls4(selected_latent_space, selected_dim_reduction_method, data, label, label_rep)

  case "label_sim":

    extract_similarities_ls3(selected_dim_reduction_method, data, label)

  case "cp":

    extract_similarities_ls2(data, label)


{'image_id': 2641, 'label': 46, 'distance': 0.013618215122607105}
{'image_id': 1686, 'label': 16, 'distance': 0.015215365128880378}
{'image_id': 2310, 'label': 35, 'distance': 0.015383486193179943}
{'image_id': 3781, 'label': 84, 'distance': 0.01541886635507712}
{'image_id': 1483, 'label': 11, 'distance': 0.015474891099448796}
{'image_id': 2719, 'label': 48, 'distance': 0.01960489858697963}
{'image_id': 3787, 'label': 85, 'distance': 0.02006387165132467}
{'image_id': 3877, 'label': 87, 'distance': 0.02050382578938892}
{'image_id': 3719, 'label': 82, 'distance': 0.02293235381986182}
{'image_id': 3403, 'label': 70, 'distance': 0.024912695992711693}
